### Requirements/Libraries


In [5]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import owlrl

### Query local Playground dataset

In [2]:
#Loads KG
g = Graph()
g.parse("./data/playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

#Same but with loading a query from a file
print("Query results, Females (query from file):")
query_file = "./data/query.txt"
query = open(query_file, 'r').read()
    
qres = g.query(query)
    
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'
Query results, Females (query from file):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'



### Query remote dataset via SPARQL Enpoint

In [3]:
#Remote service
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
LIMIT 10
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (limit to 10):")
i=0
for result in results["results"]["bindings"]:
    i+=1
    #Prints individual results 
    print(result["x"]["value"])
  

Stars co-starring Johnny Depp (limit to 10):

Abigail Breslin
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Aleksandr Kuznetsov
Alfred Molina
Alison Sudol


### Reasoning with OWL 2 RL

In [6]:
g2 = Graph()
    
g2.parse("data/example-owl2rl.ttl", format="ttl")
    
print("Loaded '" + str(len(g2)) + "' triples.")
    
#Performs OWL 2 RL reasoning: expangs graph g2
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=True, datatype_axioms=False).expand(g2)  
    
print("Rules after inference: '" + str(len(g2)) + "' triples.")
        
print("Saving extended graph")
g2.serialize(destination='../student-codes-data/example-owl2rl-extended.ttl', format='ttl')

Loaded '27' triples.
Rules after inference: '1116' triples.
Saving extended graph


<Graph identifier=N292a3f7757b947a9a159eb1e3c25fbe6 (<class 'rdflib.graph.Graph'>)>